In [1]:
!nvidia-smi

Sat Jul 26 21:40:48 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:CA:00.0 Off |                    0 |
| N/A   37C    P0              53W / 400W |      0MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Via API

In [38]:
import requests 

req = requests.post(
    "https://www.neuronpedia.org/api/steer-chat",
    headers={
      "Content-Type": "application/json"
    },
    json={
      "defaultChatMessages": [
        {
          "role": "user",
          "content": "hi"
        }
      ],
      "steeredChatMessages": [
        {
          "role": "user",
          "content": "hi"
        }
      ],
      "modelId": "gemma-2-9b-it",
      "features": [
        {
          "modelId": "gemma-2-9b-it",
          "layer": "9-gemmascope-res-131k",
          "index": 62610,
          "strength": 48
        }
      ],
      "temperature": 0.5,
      "n_tokens": 48,
      "freq_penalty": 2,
      "seed": 16,
      "strength_multiplier": 4,
      "steer_special_tokens": True
    }
)

data = req.json()
print(data)

{'STEERED': {'raw': '<bos><start_of_turn>user\nhi<end_of_turn>\n<start_of_turn>model\nMeow! 👋  How can I help you today?<end_of_turn><eos>', 'chatTemplate': [{'content': 'hi', 'role': 'user'}, {'content': 'Meow! 👋  How can I help you today?', 'role': 'model'}]}, 'DEFAULT': {'raw': '<bos><start_of_turn>user\nhi<end_of_turn>\n<start_of_turn>model\nHello! 👋\n\nHow can I help you today? 😊<end_of_turn><eos>', 'chatTemplate': [{'content': 'hi', 'role': 'user'}, {'content': 'Hello! 👋\n\nHow can I help you today? 😊', 'role': 'model'}]}, 'id': 'cm3j3ci17002zem3pxl7zg5jy', 'shareUrl': 'https://www.neuronpedia.org/steer/cm3j3ci17002zem3pxl7zg5jy', 'limit': '294', 'settings': {'temperature': 0.5, 'n_tokens': 48, 'freq_penalty': 2, 'seed': 16, 'strength_multiplier': 4, 'steer_special_tokens': True, 'steer_method': 'SIMPLE_ADDITIVE'}}


HTTP 500
{
  "message": "Unknown Error"
}


# Chat completion - RES - API 

In [11]:
# ONLY WORKS WITH RES/MLP CHAT COMPLETION
import requests 

req = requests.post("https://www.neuronpedia.org/api/steer",
    headers={
      "Content-Type": "application/json"
    },
    json={
      "prompt": "How are you feelings",
      "modelId": "gemma-2-2b",
      "features": [
        {
          "modelId": "gemma-2-2b",
          "layer": "6-gemmascope-res-16k",
          "index": 4948,
          "strength": 0
        }
      ],
      "temperature": 0.5,
      "n_tokens": 48,
      "freq_penalty": 2,
      "seed": 16,
      "strength_multiplier": 4
    }
)

data = req.json()
print(data)
print(data["STEERED"])
print()
print(data["DEFAULT"])

SyntaxError: invalid syntax (523861238.py, line 17)

# BY WEIGHTS

In [71]:
from sae_lens import SAE
import torch

# your initial template
model = HookedSAETransformer.from_pretrained("gpt2-small", device="cuda")
sae = SAE.from_pretrained(
    release="gpt2-small-res-jb",
    sae_id="blocks.7.hook_resid_pre",  # this is layer 7's residual-pre hook
    device="cuda",
)[0]
prompt = " I am happy, what should I do now"  # note the leading space for GPT‑2
logits, activation_cache = model.run_with_cache(prompt, prepend_bos=True)

hook_point = "blocks.7.hook_resid_pre"                  # e.g. "blocks.7.hook_resid_pre"
latent_activations = sae.encode(activation_cache[hook_point])
# pick the most activated feature index
top_idx = torch.topk(latent_activations, k=1).indices.item()

# decode the SAE direction back to model space
steering_vector = sae.W_dec[top_idx]  # shape [hidden_size]
steer_coeff = 0.3  # tune this value; smaller values give subtler effects
def steering_hook(resid_pre: torch.Tensor, hook):
    # resid_pre has shape [batch, sequence_length, hidden_size]
    # broadcast the steering vector across the batch and sequence:
    resid_pre += steer_coeff * steering_vector
    return resid_pre
from functools import partial

# wrap the steering hook with the correct hook name
hook_list = [(hook_point, steering_hook)]

# generate a steered completion for another prompt
with model.hooks(fwd_hooks=hook_list):
    output_tokens = model.generate(
        "Tell me a joke about animals",    # any prompt you want to steer
        max_new_tokens=50,
        temperature=1.0,
        top_p=0.9,
    )

print(model.to_string(output_tokens))


Loaded pretrained model gpt2-small into HookedTransformer


RuntimeError: a Tensor with 10 elements cannot be converted to Scalar

In [73]:
import torch
from sae_lens import SAE, HookedSAETransformer

# --- 1. Model and SAE Loading (Updated for GPT-2) ---
MODEL_NAME = "gpt2-small"
# Use the SAE release specifically for gpt2-small
RELEASE = "gpt2-small-res-jb"
# We'll use the SAE trained on layer 9. GPT-2 Small has 12 layers (0-11).
SAE_LAYER = 9
# The ID format for this release is different
SAE_ID = f"blocks.{SAE_LAYER}.hook_resid_pre"

print("Loading model...")
model = HookedSAETransformer.from_pretrained(MODEL_NAME, device="cuda")
print("Loading SAE...")

# Unpack the tuple to get the SAE object.
sae, sae_cfg, sae_sparsity = SAE.from_pretrained(
    release=RELEASE,
    sae_id=SAE_ID,
    device="cuda",
)
print("Setup complete.")


# --- 2. Steering Configuration ---
PROMPT = "Their first meeting was at a cafe in Paris. From that day on, they knew"
# New feature index for the gpt2-small layer 9 SAE.
FEATURE_INDEX = 4587 # A feature related to "romance/relationships"
# For best results, we'll steer at the same layer the SAE was trained on.
STEERING_LAYER = 9
COEFFICIENT = 20.0 # Adjusted for the new model/feature


# --- 3. Get the Steering Vector ---
steering_vector = sae.W_dec[FEATURE_INDEX] * COEFFICIENT
steering_vector = steering_vector.to(model.device)


# --- 4. Define the Hook Function ---
hook_point = f"blocks.{STEERING_LAYER}.hook_resid_pre"

def steering_hook(resid_pre, hook):
    """Adds the steering vector to the residual stream of the last token."""
    if resid_pre.shape[1] > 0:
        resid_pre[:, -1, :] += steering_vector
    return resid_pre


# --- 5. Generate Steered and Unsteered Text ---
print("\n--- Generating Steered Output (GPT-2) ---")
with model.hooks(fwd_hooks=[(hook_point, steering_hook)]):
    steered_output = model.generate(
        PROMPT,
        max_new_tokens=25,
        temperature=0.7,
        do_sample=True,
    )
print(steered_output)

print("\n--- Generating Unsteered Output (for comparison) ---")
unsteered_output = model.generate(
    PROMPT,
    max_new_tokens=25,
    temperature=0.7,
    do_sample=True,
)
print(unsteered_output)

Loading model...
Loaded pretrained model gpt2-small into HookedTransformer
Loading SAE...


cfg.json: 0.00B [00:00, ?B/s]

sae_weights.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

sparsity.safetensors:   0%|          | 0.00/98.4k [00:00<?, ?B/s]

Setup complete.


AttributeError: 'HookedSAETransformer' object has no attribute 'device'

In [44]:
import gc
import torch
gc.collect() # Python's garbage collector
torch.cuda.empty_cache()

In [60]:
# from transformer_lens import HookedTransformer
from sae_lens import SAE, HookedSAETransformer

model = HookedSAETransformer.from_pretrained("gpt2-small", device="cuda")
sae, sae_cfg, _ = SAE.from_pretrained(
    release="gpt2-small-res-jb",  # <- Release name
    sae_id="blocks.7.hook_resid_pre",  # <- SAE id (not always a hook point!)
    device="cuda",
)

# ==========================================================================
# SAE_TYPE = "res"    # res/ mlp/ att
# MODEL_NAME = "gemma-2-2b"
# RELEASE = f"gemma-scope-2b-pt-{SAE_TYPE}-canonical"
# LAYER = 25
# SAE_ID = f"layer_{LAYER}/width_16k/canonical"

# model = HookedSAETransformer.from_pretrained(MODEL_NAME, device="cuda")
# sae, cfg, _ = SAE.from_pretrained(
#     release=RELEASE,  # <- Release name
#     sae_id=SAE_ID,  # <- SAE id (not always a hook point!)
#     device="cuda",
# )

Loaded pretrained model gpt2-small into HookedTransformer


In [62]:
sae_cfg

{'model_name': 'gpt2-small',
 'hook_point': 'blocks.7.hook_resid_pre',
 'hook_point_layer': 7,
 'hook_point_head_index': None,
 'dataset_path': 'Skylion007/openwebtext',
 'is_dataset_tokenized': False,
 'context_size': 128,
 'use_cached_activations': False,
 'cached_activations_path': 'activations/Skylion007_openwebtext/gpt2-small/blocks.7.hook_resid_pre',
 'd_in': 768,
 'n_batches_in_buffer': 128,
 'total_training_tokens': 300000000,
 'store_batch_size': 32,
 'device': 'cuda',
 'seed': 42,
 'dtype': 'torch.float32',
 'b_dec_init_method': 'geometric_median',
 'expansion_factor': 32,
 'from_pretrained_path': None,
 'l1_coefficient': 8e-05,
 'lr': 0.0004,
 'lr_scheduler_name': None,
 'lr_warm_up_steps': 5000,
 'train_batch_size': 4096,
 'use_ghost_grads': False,
 'feature_sampling_window': 1000,
 'feature_sampling_method': None,
 'resample_batches': 1028,
 'feature_reinit_scale': 0.2,
 'dead_feature_window': 5000,
 'dead_feature_estimation_method': 'no_fire',
 'dead_feature_threshold': 1

In [66]:
import torch
from functools import partial
from sae_lens import SAE, HookedSAETransformer

# # --- Configuration ---
# MODEL_NAME = "gemma-2-2b"
# SAE_TYPE   = "res"                  # residual-stream SAE
# LAYER      = 0                      # layer to steer
# RELEASE    = f"gemma-scope-2b-pt-{SAE_TYPE}-canonical"
# SAE_ID     = f"layer_{LAYER}/width_16k/canonical"

# # Choose a feature to steer – find an index on Neuronpedia for layer 0
# feature_id = 0      # <-- replace with a real feature index
# strength   = 4.0    # multiplier; positive pushes towards the feature
# pos        = None   # None => apply at all token positions, or use an int index

# # --- Load model and SAE ---
# # HookedSAETransformer acts like HookedTransformer but can splice in SAEs:contentReference[oaicite:0]{index=0}.
# model: HookedSAETransformer = HookedSAETransformer.from_pretrained(MODEL_NAME, device="cuda")

# # SAE.from_pretrained returns (sae, sae_cfg, metadata)
# sae, sae_cfg, _ = SAE.from_pretrained(
#     release=RELEASE,
#     sae_id=SAE_ID,
#     device="cuda",
# )

# Determine the hook name for this SAE (e.g. "blocks.0.hook_resid_pre")
sae_layer_name = sae_cfg.metadata.hook_name
print("SAE attaches to hook:", sae_layer_name)

# --- Steering hook function ---
def multiply_feature(sae_acts: torch.Tensor, hook, pos: int, feature_id: int, strength: float):
    """
    Multiply a given SAE latent feature by (1 + strength) either at one position or all positions.
    sae_acts has shape [batch, seq_len, d_sae].
    """
    if pos is None:
        sae_acts[:, :, feature_id] *= (1.0 + strength)
    else:
        sae_acts[:, pos, feature_id] *= (1.0 + strength)
    return sae_acts

# --- Run with steering ---
prompt = "The most iconic structure in London is"  # your prompt here
tokens = model.to_tokens(prompt, prepend_bos=True).to(model.cfg.device)

# Use run_with_hooks_with_saes to splice in the SAE for this forward pass and modify the latent
# activations via a forward hook:contentReference[oaicite:1]{index=1}.
logits = model.run_with_hooks_with_saes(
    tokens,
    saes=[sae],  # list of SAEs to splice in
    fwd_hooks=[
        (
            sae_layer_name + ".hook_sae_acts_post",  # internal SAE activation hook
            partial(multiply_feature, pos=pos, feature_id=feature_id, strength=strength),
        )
    ],
)

# Get the next-token prediction from the steered logits
next_token = logits[:, -1, :].argmax(dim=-1)
steered_output = model.to_string(next_token)[0]
print("Steered next token:", steered_output)

# If you want to keep generating more tokens, loop by appending `next_token` to `tokens`
# and calling model.run_with_hooks_with_saes again for each new step.


SAE attaches to hook: blocks.7.hook_resid_pre
Steered next token:  


In [25]:
# instantiate an object to hold activations from a dataset
from sae_lens import ActivationsStore

# a convenient way to instantiate an activation store is to use the from_sae method
activation_store = ActivationsStore.from_sae(
    model=model,
    dataset=sae.cfg.metadata.dataset_path,
    sae=sae,
    streaming=True,
    # fairly conservative parameters here so can use same for larger
    # models without running out of memory.
    store_batch_size_prompts=8,
    train_batch_size_tokens=4096,
    n_batches_in_buffer=32,
    device=device,
)

# Working 1 

In [17]:
import torch
from functools import partial
from sae_lens import SAE, HookedSAETransformer

# --- Configuration ---
MODEL_NAME = "gpt2-small"
RELEASE    = "gpt2-small-res-jb"
SAE_ID     = "blocks.7.hook_resid_pre"
DEVICE     = "cuda"


# --- Load model and SAE ---
model = HookedSAETransformer.from_pretrained(MODEL_NAME, device=DEVICE)
sae, sae_cfg, _ = SAE.from_pretrained(
    release=RELEASE,
    sae_id=SAE_ID,
    device=DEVICE,
)

# sae_cfg is a dict, so get the hook name under "hook_point"
sae_layer_name = sae_cfg["hook_point"]
hook_name = sae_cfg["hook_point"]
print("SAE attaches to hook:", sae_layer_name)


# --- Steering parameters ---
# prompt     = "The most iconic structure in London is"
prompt     = "I am feeling good"

feature_id = 2     # ← *replace* with a real feature index from Neuronpedia
feature_id = 1         # choose an actually dormant or live feature

strength   = 20.0       # multiplier (e.g. 4.0 means 5× the original activation)
pos        = None      # None=all token positions, or an int for a single position
max_new_tokens = 64

Loaded pretrained model gpt2-small into HookedTransformer
SAE attaches to hook: blocks.7.hook_resid_pre


In [18]:

# --- Hook function to multiply one SAE feature ---
def multiply_feature(sae_acts: torch.Tensor, hook, pos: int, feature_id: int, strength: float):
    if pos is None:
        sae_acts[:, :, feature_id] *= (1.0 + strength)
    else:
        sae_acts[:, pos, feature_id] *= (1.0 + strength)
    return sae_acts

# --- Steered generation loop ---
def steer_generate(prompt: str, max_new_tokens: int = 20) -> str:
    generated = model.to_tokens(prompt, prepend_bos=True).to(DEVICE)
    for _ in range(max_new_tokens):
        logits = model.run_with_hooks_with_saes(
            generated,
            saes=[sae],
            fwd_hooks=[
                (
                    sae_layer_name + ".hook_sae_acts_post",
                    partial(multiply_feature, pos=pos, feature_id=feature_id, strength=strength),
                )
            ],
        )
        next_tok = logits[:, -1, :].argmax(dim=-1).unsqueeze(-1)
        generated = torch.cat((generated, next_tok), dim=1)
    return model.to_string(generated)[0]

# --- Example usage ---
if __name__ == "__main__":
    # out = steer_generate("The most iconic structure in London is")
    out = steer_generate(prompt, max_new_tokens=max_new_tokens)
    print("Steered output:\n", out)


Steered output:
 <|endoftext|>I am feeling good about this. I am not sure if I am feeling good about this. I am not sure if I am feeling good about this.

I am not sure if I am feeling good about this. I am not sure if I am feeling good about this.

I am not sure if I am feeling good


# Working 2

In [19]:
# 2) Hook to inject
def inject_feature(sae_acts, hook, pos, feature_id, strength):
    if pos is None:
        sae_acts[:, :, feature_id] = strength
    else:
        sae_acts[:, pos, feature_id] = strength
    return sae_acts

# 3) Steered generation
def steer_generate(prompt, feature_id, strength, max_new_tokens=10):
    tokens = model.to_tokens(prompt, prepend_bos=True).to("cuda")
    for _ in range(max_new_tokens):
        logits = model.run_with_hooks_with_saes(
            tokens,
            saes=[sae],
            fwd_hooks=[
                (
                    hook_name + ".hook_sae_acts_post",
                    partial(inject_feature, pos=None, feature_id=feature_id, strength=strength),
                )
            ],
        )
        nxt = logits[:, -1, :].argmax(dim=-1).unsqueeze(-1)
        tokens = torch.cat((tokens, nxt), dim=1)
    return model.to_string(tokens)[0]

# 4) Try it
baseline = model.generate(prompt, max_new_tokens=20, temperature=0.7, do_sample=True)
print(baseline)
print(steer_generate(prompt, feature_id, strength, max_new_tokens=max_new_tokens))


  0%|          | 0/20 [00:00<?, ?it/s]

I am feeling good and happy. I see that most of my friends are doing well too.

I am happy
<|endoftext|>I am feeling good about this. I am not sure if I can get a job, but I am sure that I am doing my job. I am not going to be able to get a job. I am not going to be able to get a job. I am not going to be able to get a job. I am not


In [22]:
import torch
from functools import partial
from sae_lens import SAE, HookedSAETransformer

# --- Configuration ---
MODEL_NAME = "gpt2-small"
RELEASE    = "gpt2-small-res-jb"
SAE_ID     = "blocks.7.hook_resid_pre"
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

# --- Load model and SAE ---
model = HookedSAETransformer.from_pretrained(MODEL_NAME, device=DEVICE)
sae, sae_cfg, *_ = SAE.from_pretrained(
    release=RELEASE,
    sae_id=SAE_ID,
    device=DEVICE,
)

hook_name = sae_cfg["hook_point"]
print(f"SAE attaches to hook: {hook_name}")


Loaded pretrained model gpt2-small into HookedTransformer
SAE attaches to hook: blocks.7.hook_resid_pre


In [23]:

# =============================================================================
# NEURONPEDIA-STYLE STEERING: ADDITION-BASED
# =============================================================================

def add_feature_steering(sae_acts: torch.Tensor, hook, pos: int, feature_id: int, strength: float):
    """
    Neuronpedia-style feature steering using ADDITION instead of multiplication.
    This adds a constant value to the feature activation.
    
    Args:
        sae_acts: SAE activations tensor [batch, seq_len, n_features]
        hook: TransformerLens hook object
        pos: Position to steer (None for all positions, int for specific position)
        feature_id: Which SAE feature to steer
        strength: How much to ADD to the feature activation
    """
    if pos is None:
        # Steer all positions
        sae_acts[:, :, feature_id] += strength
    else:
        # Steer specific position
        sae_acts[:, pos, feature_id] += strength
    return sae_acts

def neuronpedia_steer_generate(
    prompt: str, 
    feature_id: int, 
    strength: float, 
    pos: int = None,
    max_new_tokens: int = 20,
    temperature: float = 0.7
) -> str:
    """
    Generate text with Neuronpedia-style SAE feature steering.
    
    Args:
        prompt: Input text prompt
        feature_id: SAE feature index to steer
        strength: Steering strength (positive = increase, negative = decrease)
        pos: Token position to steer (None = all positions, -1 = last token only)
        max_new_tokens: Number of tokens to generate
        temperature: Sampling temperature (0 = greedy, higher = more random)
    
    Returns:
        Generated text string
    """
    tokens = model.to_tokens(prompt, prepend_bos=True).to(DEVICE)
    
    for _ in range(max_new_tokens):
        # Run model with SAE and steering hook
        logits = model.run_with_hooks_with_saes(
            tokens, 
            saes=[sae],
            fwd_hooks=[
                (
                    hook_name + ".hook_sae_acts_post",
                    partial(add_feature_steering, pos=pos, feature_id=feature_id, strength=strength)
                )
            ]
        )
        
        # Sample next token
        if temperature == 0:
            next_token = logits[:, -1, :].argmax(dim=-1).unsqueeze(-1)
        else:
            probs = torch.softmax(logits[:, -1, :] / temperature, dim=-1)
            next_token = torch.multinomial(probs, 1)
        
        tokens = torch.cat([tokens, next_token], dim=1)
    
    return model.to_string(tokens)[0]


In [47]:

# =============================================================================
# EXAMPLE USAGE
# =============================================================================

if __name__ == "__main__":
    prompt = "I am feeling good"
    feature_id = 1000  # Replace with actual feature ID from Neuronpedia
    
    # print("=== BASELINE (No Steering) ===")
    # baseline = model.generate(prompt, max_new_tokens=64, temperature=0.5, do_sample=True)
    # print(f"Baseline: {baseline}")

    # TOKENS: 64, TEMP: 0.5, FREQ PENALTY: 1, STRENGTH MULTIPLE: 1, MANUAL SEED: 16
    neuronpedia_settings = {
        'max_new_tokens': 64,
        'temperature': 0.5, 
        'freq_penalty': 1.0,  # No frequency penalty
        'seed': 16  # Manual seed from interface
    }
    print("=== BASELINE (No Steering) ===")
    # Set seed for baseline too
    # torch.manual_seed(neuronpedia_settings['seed'])
    # if torch.cuda.is_available():
    #     torch.cuda.manual_seed(neuronpedia_settings['seed'])
    baseline = model.generate(
        prompt, 
        max_new_tokens=neuronpedia_settings['max_new_tokens'], 
        temperature=neuronpedia_settings['temperature'], 
        do_sample=True
    )
    print(f"Baseline: {baseline}")


    
    print("\n=== NEURONPEDIA-STYLE STEERING (Addition) ===")
    # Positive steering - enhance the feature
    steered_pos = neuronpedia_steer_generate(
        prompt=prompt,
        feature_id=feature_id,
        strength=10.0,  # Add 10.0 to feature activation
        pos=None,       # Steer all positions
        max_new_tokens=20,
        temperature=0.7
    )
    print(f"Addition (+10.0): {steered_pos}")
    
    # Negative steering - suppress the feature
    steered_neg = neuronpedia_steer_generate(
        prompt=prompt,
        feature_id=feature_id,
        strength=-5.0,  # Subtract 5.0 from feature activation
        pos=None,
        max_new_tokens=20,
        temperature=0.7
    )
    print(f"Addition (-5.0): {steered_neg}")
    
    print("\n=== INJECTION STEERING (Set Value) ===")
    # Injection steering - force specific activation
    injected = injection_steer_generate(
        prompt=prompt,
        feature_id=feature_id,
        strength=15.0,  # Set feature activation to exactly 15.0
        pos=None,
        max_new_tokens=20,
        temperature=0.7
    )
    print(f"Injection (=15.0): {injected}")
    
    print("\n=== POSITION-SPECIFIC STEERING ===")
    # Steer only the last token position
    last_pos_steered = neuronpedia_steer_generate(
        prompt=prompt,
        feature_id=feature_id,
        strength=8.0,
        pos=-1,  # Only steer the last token
        max_new_tokens=20,
        temperature=0.5
    )
    print(f"Last position only (+8.0): {last_pos_steered}")



=== BASELINE (No Steering) ===


  0%|          | 0/64 [00:00<?, ?it/s]

Baseline: I am feeling good about my health right now. I am having trouble getting the vitamins I need. I am feeling better about my health right now. I am feeling better about my health right now. I am feeling better about my health right now. I am feeling better about my health right now. I am feeling better about my health right

=== NEURONPEDIA-STYLE STEERING (Addition) ===
Addition (+10.0): <|endoftext|>I am feeling good about my life.

I'm thinking about the 5 steps I took to become a professional.
Addition (-5.0): <|endoftext|>I am feeling good about the state of the game

So, if you have read my previous post, you will

=== INJECTION STEERING (Set Value) ===
Injection (=15.0): <|endoftext|>I am feeling good about my performance, but I have to let go from the situation I have with my mouth.


=== POSITION-SPECIFIC STEERING ===
Last position only (+8.0): <|endoftext|>I am feeling good about this. I'm really excited about this. I'm not just looking forward to this season.


# =====================================================================================

In [97]:
import torch
from sae_lens import SAE, HookedSAETransformer

# --- 1. Model and SAE Loading ---
MODEL_NAME = "gpt2-small"
RELEASE = "gpt2-small-res-jb"
SAE_LAYER = 7
SAE_ID = f"blocks.{SAE_LAYER}.hook_resid_pre"

print("Loading model...")
model = HookedSAETransformer.from_pretrained(MODEL_NAME, device="cuda")
print("Loading SAE...")

sae, sae_cfg, sae_sparsity = SAE.from_pretrained(
    release=RELEASE,
    sae_id=SAE_ID,
    device="cuda",
)
print("Setup complete.")


# --- 2. Steering Configuration ---
PROMPT = "The most iconic structure in London is"
FEATURE_INDEX = 1 # A feature related to "romance/relationships"
STEERING_LAYER = 7
COEFFICIENT = 20.0


# --- 3. Get the Steering Vector ---
steering_vector = sae.W_dec[FEATURE_INDEX] * COEFFICIENT
# CORRECTED LINE: Get the device from the model's config object.
steering_vector = steering_vector.to(model.cfg.device)


# --- 4. Define the Hook Function ---
hook_point = f"blocks.{STEERING_LAYER}.hook_resid_pre"

def steering_hook(resid_pre, hook):
    """Adds the steering vector to the residual stream of the last token."""
    if resid_pre.shape[1] > 0:
        resid_pre[:, -1, :] += steering_vector
    return resid_pre


# --- 5. Generate Steered and Unsteered Text ---
print("\n--- Generating Steered Output (GPT-2) ---")
with model.hooks(fwd_hooks=[(hook_point, steering_hook)]):
    steered_output = model.generate(
        PROMPT,
        max_new_tokens=25,
        temperature=0.7,
        do_sample=True,
    )
print(steered_output)

print("\n--- Generating Unsteered Output (for comparison) ---")
unsteered_output = model.generate(
    PROMPT,
    max_new_tokens=25,
    temperature=0.7,
    do_sample=True,
)
print(unsteered_output)

Loading model...
Loaded pretrained model gpt2-small into HookedTransformer
Loading SAE...
Setup complete.

--- Generating Steered Output (GPT-2) ---


  0%|          | 0/25 [00:00<?, ?it/s]

The most iconic structure in London is now being demolished.

Crackhead Building, which is believed to still be in use in London, is the tallest

--- Generating Unsteered Output (for comparison) ---


  0%|          | 0/25 [00:00<?, ?it/s]

The most iconic structure in London is the National Building Museum. It's a white building in the middle of the city where the National Gallery of Art is located and


In [93]:
import torch
from tqdm import tqdm
from functools import partial
import re
from sae_lens import SAE, HookedSAETransformer, ActivationsStore

# --- Configuration ---
MODEL_NAME = "gpt2-small"
RELEASE    = "gpt2-small-res-jb"
SAE_ID     = "blocks.7.hook_resid_pre"
DEVICE     = "cuda"

# --- Load model and SAE ---
model = HookedSAETransformer.from_pretrained(MODEL_NAME, device=DEVICE)
sae, sae_cfg, _ = SAE.from_pretrained(
    release=RELEASE,
    sae_id=SAE_ID,
    device=DEVICE,
)

# Extract hook point and prepend_bos flag
hook_point = sae_cfg["hook_point"]               # e.g. "blocks.7.hook_resid_pre"
prepend_bos = sae_cfg.get("prepend_bos", True)

# --- Activation Store Setup ---
activation_store = ActivationsStore.from_sae(
    model=model,
    sae=sae,
    # dataset=sae_cfg["dataset_path"],
    dataset="openwebtext",
    streaming=True,
    store_batch_size_prompts=8,
    train_batch_size_tokens=4096,
    n_batches_in_buffer=32,
    device=DEVICE,
)

def find_max_activation(model, sae, sae_cfg, activation_store, feature_idx, num_batches=100):
    """
    Find the maximum absolute activation for `feature_idx` across samples.
    """
    max_act = 0.0
    # parse layer number from hook_point
    layer = int(re.search(r"\.(\d+)\.", hook_point).group(1))
    for _ in tqdm(range(num_batches), desc="Finding max activation"):
        tokens = activation_store.get_batch_tokens().to(DEVICE)
        # run up to just after the hook layer
        _, cache = model.run_with_cache(
            tokens,
            stop_at_layer=layer + 1,
            names_filter=[hook_point],
        )
        sae_in = cache[hook_point]                      # [batch, seq, d_in]
        lats, _ = sae.encode(sae_in)                    # [batch, seq, d_sae], _
        lats = lats.reshape(-1, lats.shape[-1])         # flatten to [batch*seq, d_sae]
        batch_max = lats[:, feature_idx].abs().max().item()
        max_act = max(max_act, batch_max)
    return max_act

def steering_hook(activations, hook, steering_vector, steering_strength, max_act):
    """
    Inject a clamped steering vector into SAE activations.
    """
    sv = steering_vector.to(activations.device)
    sv = torch.clamp(sv, -max_act, max_act)
    return activations + steering_strength * sv.unsqueeze(0).unsqueeze(0)

def generate_with_steering(
    model,
    sae,
    sae_cfg,
    prompt,
    steering_feature,
    max_act,
    steering_strength=1.0,
    max_new_tokens=95,
):
    device = next(model.parameters()).device
    input_ids = model.to_tokens(prompt, prepend_bos=prepend_bos).to(device)
    steering_vector = sae.W_dec[steering_feature].to(device)
    hook = hook_point + ".hook_sae_acts_post"
    hook_fn = partial(
        steering_hook,
        steering_vector=steering_vector,
        steering_strength=steering_strength,
        max_act=max_act,
    )
    with model.hooks(fwd_hooks=[(hook, hook_fn)]):
        out_ids = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            stop_at_eos=True,
            prepend_bos=prepend_bos,
        )
    return model.tokenizer.decode(out_ids[0])

# --- Main Execution ---
if __name__ == "__main__":
    prompt = "Once upon a time"
    steering_feature = 20115  # pick a feature index (e.g. top from Neuronpedia)
    max_act = find_max_activation(model, sae, sae_cfg, activation_store, steering_feature, num_batches=50)
    print(f"Max activation for feature {steering_feature}: {max_act:.4f}\n")

    # Generate without steering
    tokens = model.to_tokens(prompt, prepend_bos=prepend_bos).to(DEVICE)
    normal_ids = model.generate(tokens, max_new_tokens=95, temperature=0.7, top_p=0.9, stop_at_eos=True, prepend_bos=prepend_bos)
    normal_text = model.tokenizer.decode(normal_ids[0])
    print("Normal generation:\n", normal_text, "\n")

    # Generate with steering
    steered_text = generate_with_steering(
        model, sae, sae_cfg, prompt, steering_feature, max_act, steering_strength=2.0, max_new_tokens=95
    )
    print("Steered generation:\n", steered_text)


Loaded pretrained model gpt2-small into HookedTransformer


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

# FINAL

In [39]:
import torch
from functools import partial
from sae_lens import SAE, HookedSAETransformer

# --- Configuration ---
MODEL_NAME = "gpt2-small"
RELEASE    = "gpt2-small-res-jb"
SAE_ID     = "blocks.7.hook_resid_pre"
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

# --- Load model and SAE ---
model = HookedSAETransformer.from_pretrained(MODEL_NAME, device=DEVICE)
sae, sae_cfg, *_ = SAE.from_pretrained(
    release=RELEASE,
    sae_id=SAE_ID,
    device=DEVICE,
)

hook_name = sae_cfg["hook_point"]
print(f"SAE attaches to hook: {hook_name}")

# =============================================================================
# NEURONPEDIA-STYLE STEERING: ADDITION-BASED
# =============================================================================

def add_feature_steering(sae_acts: torch.Tensor, hook, pos: int, feature_id: int, strength: float):
    """
    Neuronpedia-style feature steering using ADDITION instead of multiplication.
    This adds a constant value to the feature activation.
    
    Args:
        sae_acts: SAE activations tensor [batch, seq_len, n_features]
        hook: TransformerLens hook object
        pos: Position to steer (None for all positions, int for specific position)
        feature_id: Which SAE feature to steer
        strength: How much to ADD to the feature activation
    """
    if pos is None:
        # Steer all positions
        sae_acts[:, :, feature_id] += strength
    else:
        # Steer specific position
        sae_acts[:, pos, feature_id] += strength
    return sae_acts

def neuronpedia_steer_generate(
    prompt: str, 
    feature_id: int, 
    strength: float, 
    pos: int = None,
    max_new_tokens: int = 20,
    temperature: float = 0.7,
    freq_penalty: float = 1.0,
    seed: int = None
) -> str:
    """
    Generate text with Neuronpedia-style SAE feature steering.
    
    Args:
        prompt: Input text prompt
        feature_id: SAE feature index to steer
        strength: Steering strength (positive = increase, negative = decrease)
        pos: Token position to steer (None = all positions, -1 = last token only)
        max_new_tokens: Number of tokens to generate
        temperature: Sampling temperature (0 = greedy, higher = more random)
        freq_penalty: Frequency penalty (1.0 = no penalty, >1.0 = penalize repetition)
        seed: Random seed for reproducibility
    
    Returns:
        Generated text string
    """
    # Set random seed if provided
    if seed is not None:
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
    
    tokens = model.to_tokens(prompt, prepend_bos=True).to(DEVICE)
    
    # Track token frequencies for frequency penalty
    token_counts = {}
    
    for step in range(max_new_tokens):
        # Run model with SAE and steering hook
        logits = model.run_with_hooks_with_saes(
            tokens, 
            saes=[sae],
            fwd_hooks=[
                (
                    hook_name + ".hook_sae_acts_post",
                    partial(add_feature_steering, pos=pos, feature_id=feature_id, strength=strength)
                )
            ]
        )
        
        # Apply frequency penalty
        if freq_penalty != 1.0:
            for token_id, count in token_counts.items():
                logits[0, -1, token_id] /= (freq_penalty ** count)
        
        # Sample next token
        if temperature == 0:
            next_token = logits[:, -1, :].argmax(dim=-1).unsqueeze(-1)
        else:
            probs = torch.softmax(logits[:, -1, :] / temperature, dim=-1)
            next_token = torch.multinomial(probs, 1)
        
        # Update token count for frequency penalty
        token_id = next_token.item()
        token_counts[token_id] = token_counts.get(token_id, 0) + 1
        
        tokens = torch.cat([tokens, next_token], dim=1)
    
    return model.to_string(tokens)[0]

# =============================================================================
# INJECTION-BASED STEERING (Alternative approach)
# =============================================================================

def inject_feature_activation(sae_acts: torch.Tensor, hook, pos: int, feature_id: int, strength: float):
    """
    Injection-based steering: SET the feature activation to a specific value.
    This completely overwrites the natural activation.
    """
    if pos is None:
        sae_acts[:, :, feature_id] = strength
    else:
        sae_acts[:, pos, feature_id] = strength
    return sae_acts

def injection_steer_generate(
    prompt: str, 
    feature_id: int, 
    strength: float, 
    pos: int = None,
    max_new_tokens: int = 20,
    temperature: float = 0.7
) -> str:
    """Generate text with injection-based SAE feature steering."""
    tokens = model.to_tokens(prompt, prepend_bos=True).to(DEVICE)
    
    for _ in range(max_new_tokens):
        logits = model.run_with_hooks_with_saes(
            tokens, 
            saes=[sae],
            fwd_hooks=[
                (
                    hook_name + ".hook_sae_acts_post",
                    partial(inject_feature_activation, pos=pos, feature_id=feature_id, strength=strength)
                )
            ]
        )
        
        if temperature == 0:
            next_token = logits[:, -1, :].argmax(dim=-1).unsqueeze(-1)
        else:
            probs = torch.softmax(logits[:, -1, :] / temperature, dim=-1)
            next_token = torch.multinomial(probs, 1)
        
        tokens = torch.cat([tokens, next_token], dim=1)
    
    return model.to_string(tokens)[0]

# =============================================================================
# EXAMPLE USAGE MATCHING NEURONPEDIA SETTINGS
# =============================================================================

if __name__ == "__main__":
    prompt = "I am feeling"
    feature_id = 1000  # Replace with actual feature ID from Neuronpedia
    
    # Neuronpedia settings from the interface:
    # TOKENS: 64, TEMP: 0.5, FREQ PENALTY: 1, STRENGTH MULTIPLE: 1, MANUAL SEED: 16
    neuronpedia_settings = {
        'max_new_tokens': 64,
        'temperature': 0.5, 
        'freq_penalty': 1.0,  # No frequency penalty
        'seed': 16  # Manual seed from interface
    }
    
    print("=== BASELINE (No Steering) ===")
    # Set seed for baseline too
    torch.manual_seed(neuronpedia_settings['seed'])
    if torch.cuda.is_available():
        torch.cuda.manual_seed(neuronpedia_settings['seed'])
    
    baseline = model.generate(
        prompt, 
        max_new_tokens=neuronpedia_settings['max_new_tokens'], 
        temperature=neuronpedia_settings['temperature'], 
        do_sample=True
    )
    print(f"Baseline: {baseline}")
    
    print("\n=== NEURONPEDIA-STYLE STEERING (Exact Match) ===")
    # Feature 1000 with strength 20.0 (from interface: strength=20, multiple=1)
    steered_neuronpedia = neuronpedia_steer_generate(
        prompt=prompt,
        feature_id=feature_id,
        strength=20.0,  # Strength from interface
        pos=None,       # Steer all positions (Simple Additive mode)
        **neuronpedia_settings
    )
    print(f"Neuronpedia Match: {steered_neuronpedia}")
    
    print("\n=== TESTING DIFFERENT STRENGTHS ===")
    for strength in [5.0, 10.0, 20.0, 30.0]:
        steered = neuronpedia_steer_generate(
            prompt=prompt,
            feature_id=feature_id,
            strength=strength,
            pos=None,
            **neuronpedia_settings
        )
        print(f"Strength {strength:4.1f}: {steered}")

# =============================================================================
# NEURONPEDIA-SPECIFIC FEATURE UTILITIES
# =============================================================================

def neuronpedia_compatible_steering(
    prompt: str,
    feature_spec: str,  # e.g., "7-RES-JB:1000" 
    strength: float = 20.0,
    **kwargs
):
    """
    Steer using Neuronpedia-style feature specification.
    
    Args:
        prompt: Input text
        feature_spec: Feature specification like "7-RES-JB:1000" 
        strength: Steering strength
        **kwargs: Additional generation parameters
    """
    # Parse feature spec (simplified - assumes format "LAYER-SOURCE:FEATURE_ID")
    if ':' in feature_spec:
        source_part, feature_id = feature_spec.split(':')
        feature_id = int(feature_id)
    else:
        raise ValueError("Feature spec should be in format 'LAYER-SOURCE:FEATURE_ID'")
    
    return neuronpedia_steer_generate(
        prompt=prompt,
        feature_id=feature_id, 
        strength=strength,
        pos=None,  # Neuronpedia default
        **kwargs
    )

# =============================================================================
# FEATURE DISCOVERY UTILITIES
# =============================================================================

def find_active_features(prompt: str, top_k: int = 10):
    """
    Find the most active SAE features for a given prompt.
    Useful for discovering which features to steer.
    """
    tokens = model.to_tokens(prompt, prepend_bos=True).to(DEVICE)
    
    # Get SAE activations
    with torch.no_grad():
        _, cache = model.run_with_cache_with_saes(tokens, saes=[sae])
        sae_acts = cache[hook_name + ".hook_sae_acts_post"]
    
    # Average over sequence length and batch
    mean_acts = sae_acts.mean(dim=(0, 1))  # [n_features]
    
    # Get top-k most active features
    top_values, top_indices = torch.topk(mean_acts, top_k)
    
    print(f"\nTop {top_k} active features for prompt: '{prompt}'")
    print("Feature ID | Activation")
    print("-" * 25)
    for i, (idx, val) in enumerate(zip(top_indices, top_values)):
        print(f"{idx.item():9d} | {val.item():8.3f}")
    
    return top_indices.cpu().numpy(), top_values.cpu().numpy()

# Example: Find active features for your prompt
if __name__ == "__main__":
    find_active_features("I am feeling good", top_k=5)

Loaded pretrained model gpt2-small into HookedTransformer
SAE attaches to hook: blocks.7.hook_resid_pre
=== BASELINE (No Steering) ===


  0%|          | 0/64 [00:00<?, ?it/s]

Baseline: I am feeling a bit of a little bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of a bit of

=== NEURONPEDIA-STYLE STEERING (Exact Match) ===
Neuronpedia Match: <|endoftext|>I am feeling a bit of a break-up, and I'm glad I got my first job as a software developer for a new client. I am now an internet journalist, host, programmer, and a writer. I am also a social media manager, a writer, and a producer. I have a living, working, and

=== TESTING DIFFERENT STRENGTHS ===
Strength  5.0: <|endoftext|>I am feeling a bit of a sense of déjà vu as I feel like I'm going to be taking a break and having a lot of fun. I'll be back in the morning, but I'm still feeling a bit of a hangover. I'm a little tired of being in a job, and I know
Strength 10.0: <|endoftext|>I am feeling a bit of a break-up, and I'm glad I got my stuff back together. I hope to do some more work on th

In [50]:
import torch
from functools import partial
from sae_lens import SAE, HookedSAETransformer

# grab the decoder vector for that feature
steering_vector = sae.W_dec[1000].to(model.cfg.device)

# --- 3. Hook that adds it into the residual stream ---
def steering_hook(resid_pre, hook):
    # only add to the last token’s residual
    resid_pre[:, -1, :] += steering_vector * strength
    return resid_pre

# --- 4. Generate with & without steering ---
prompt = "I am feeling good"

# Unsteered baseline
baseline = model.generate(
    prompt,
    max_new_tokens=64,
    temperature=0.5,
    top_p=0.9,
)

# Steered via the same SAE feature
with model.hooks(fwd_hooks=[(hook_name, steering_hook)]):
    steered = model.generate(
        prompt,
        max_new_tokens=64,
        temperature=0.2,
        top_p=0.9,
    )

print("Baseline:", baseline)
print("Steered: ", steered)


  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

Baseline: I am feeling good about my relationship with my daughter. She is a very bright and beautiful girl. She has been my best friend for over two years now. She is always there for me when I need her. She is always there for me when I need her. She is always there for me when I need her. She is always
Steered:  I am feeling good about my performance. I am not a professional athlete. I am not a professional athlete. I am not a professional athlete. I am not a professional athlete. I am not a professional athlete. I am not a professional athlete. I am not a professional athlete. I am not a professional athlete. I am not a


# Pipeline

In [52]:
import torch
from functools import partial
from sae_lens import SAE, HookedSAETransformer

# --- Configuration ---
MODEL_NAME = "gpt2-small"
RELEASE    = "gpt2-small-res-jb"
SAE_ID     = "blocks.7.hook_resid_pre"
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

# --- Load model and SAE ---
model = HookedSAETransformer.from_pretrained(MODEL_NAME, device=DEVICE)
sae, sae_cfg, *_ = SAE.from_pretrained(
    release=RELEASE,
    sae_id=SAE_ID,
    device=DEVICE,
)

hook_name = sae_cfg["hook_point"]
print(f"SAE attaches to hook: {hook_name}")


Loaded pretrained model gpt2-small into HookedTransformer
SAE attaches to hook: blocks.7.hook_resid_pre


In [53]:
# =============================================================================
# NEURONPEDIA-STYLE STEERING: ADDITION-BASED
# =============================================================================

def add_feature_steering(sae_acts: torch.Tensor, hook, pos: int, feature_id: int, strength: float):
    """
    Neuronpedia-style feature steering using ADDITION instead of multiplication.
    This adds a constant value to the feature activation.
    
    Args:
        sae_acts: SAE activations tensor [batch, seq_len, n_features]
        hook: TransformerLens hook object
        pos: Position to steer (None for all positions, int for specific position)
        feature_id: Which SAE feature to steer
        strength: How much to ADD to the feature activation
    """
    if pos is None:
        # Steer all positions
        sae_acts[:, :, feature_id] += strength
    else:
        # Steer specific position
        sae_acts[:, pos, feature_id] += strength
    return sae_acts

def neuronpedia_steer_generate(
    prompt: str, 
    feature_id: int, 
    strength: float, 
    pos: int = None,
    max_new_tokens: int = 20,
    temperature: float = 0.7
) -> str:
    """
    Generate text with Neuronpedia-style SAE feature steering.
    
    Args:
        prompt: Input text prompt
        feature_id: SAE feature index to steer
        strength: Steering strength (positive = increase, negative = decrease)
        pos: Token position to steer (None = all positions, -1 = last token only)
        max_new_tokens: Number of tokens to generate
        temperature: Sampling temperature (0 = greedy, higher = more random)
    
    Returns:
        Generated text string
    """
    tokens = model.to_tokens(prompt, prepend_bos=True).to(DEVICE)
    
    for _ in range(max_new_tokens):
        # Run model with SAE and steering hook
        logits = model.run_with_hooks_with_saes(
            tokens, 
            saes=[sae],
            fwd_hooks=[
                (
                    hook_name + ".hook_sae_acts_post",
                    partial(add_feature_steering, pos=pos, feature_id=feature_id, strength=strength)
                )
            ]
        )
        
        # Sample next token
        if temperature == 0:
            next_token = logits[:, -1, :].argmax(dim=-1).unsqueeze(-1)
        else:
            probs = torch.softmax(logits[:, -1, :] / temperature, dim=-1)
            next_token = torch.multinomial(probs, 1)
        
        tokens = torch.cat([tokens, next_token], dim=1)
    
    return model.to_string(tokens)[0]


In [56]:

if __name__ == "__main__":
    prompt = "I am feeling good"
    feature_id = 1000  


    neuronpedia_settings = {
        'max_new_tokens': 64,
        'temperature': 0.5, 
        'freq_penalty': 1.0,  # No frequency penalty
        'seed': 16  # Manual seed from interface
    }
    print("=== BASELINE (No Steering) ===")
    baseline = model.generate(
        prompt, 
        max_new_tokens=neuronpedia_settings['max_new_tokens'], 
        temperature=neuronpedia_settings['temperature'], 
        do_sample=True
    )
    print(f"Baseline: {baseline}")


    STRENGTH = 10
    MAX_NEW_TOKENS = 64
    TEMPERATURE = 0.5
    
    print("\n=== NEURONPEDIA-STYLE STEERING (Addition) ===")
    # Positive steering - enhance the feature
    steered_pos = neuronpedia_steer_generate(
        prompt=prompt,
        feature_id=feature_id,
        strength=5.0,  # Add 10.0 to feature activation
        pos=None,       # Steer all positions
        max_new_tokens=MAX_NEW_TOKENS,
        temperature=TEMPERATURE
    )
    print(f"Addition (+10.0): {steered_pos}")
    
    # Negative steering - suppress the feature
    steered_neg = neuronpedia_steer_generate(
        prompt=prompt,
        feature_id=feature_id,
        strength=-5.0,  # Subtract 5.0 from feature activation
        pos=None,
        max_new_tokens=MAX_NEW_TOKENS,
        temperature=TEMPERATURE
    )
    print(f"Addition (-5.0): {steered_neg}")
    

=== BASELINE (No Steering) ===


  0%|          | 0/64 [00:00<?, ?it/s]

Baseline: I am feeling good. I am feeling good.

I am feeling good.

I am feeling good.

I am feeling good.

I am feeling good.

I am feeling good.

I am feeling good.

I am feeling good.

I am feeling good.



=== NEURONPEDIA-STYLE STEERING (Addition) ===
Addition (+10.0): <|endoftext|>I am feeling good about it, but I really don't want to feel bad about it. I'm not going to get in trouble. I don't care about it. I don't care about what's going on in my life. I don't care about feeling good about it. I don't care about what's going on in
Addition (-5.0): <|endoftext|>I am feeling good about this, I am not happy about it. I am not happy about it. I am not happy about my wife's feelings. I am not happy about her feelings. I am not happy about her feelings. I am not happy about my husband's feelings. I am happy about her feelings. I am not happy


# For a list of neurons steer 

In [26]:
import pandas as pd
path = "./datasets/sad_vs_sad.csv"
df = pd.read_csv(path)
df = df[df["llm_decision"] == "yes"]
df.head()

,type,layer,class1,class2,Neuron_ID,Contribution,Explanation,llm_decision
196,att,22,sad,sad,9418,179776.0,attends to emotional tokens from sorrow-relat...,yes
735,mlp,22,sad,sad,3164,16384.0,references to the concept of challenges or con...,yes
753,mlp,18,sad,sad,4011,15376.0,expressions of sadness or loss,yes
803,mlp,17,sad,sad,8719,11881.0,references to death and dying,yes
814,res,15,sad,sad,13368,11664.0,elements related to internal strength and emot...,yes


In [27]:
sample = "I'm waiting to hear back from my doctor about my test results."

In [66]:
import torch
from functools import partial
from sae_lens import SAE, HookedSAETransformer

DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "gpt2-small"
RELEASE    = "gpt2-small-res-jb"
SAE_ID     = "blocks.7.hook_resid_pre"

model = HookedSAETransformer.from_pretrained(MODEL_NAME, device=DEVICE)
sae, sae_cfg, _ = SAE.from_pretrained(release=RELEASE, sae_id=SAE_ID, device=DEVICE)

hook_point = sae_cfg["hook_point"]       # e.g. "blocks.7.hook_resid_pre"
feature_id = 1000
strength   = 1.0

# get the decoder vector
v = sae.W_dec[feature_id].to(DEVICE)

def simple_additive(resid_pre, hook, steering_vector, strength):
    resid_pre[:, -1, :] += steering_vector * strength
    return resid_pre

def steer_simple_add(prompt, strength, max_new_tokens=64, temperature=0.5):
    # -- set seed globally --
    torch.manual_seed(16)
    if DEVICE=="cuda":
        torch.cuda.manual_seed(16)

    tokens = model.to_tokens(prompt, prepend_bos=True).to(DEVICE)
    fwd_hook = (hook_point,
                partial(simple_additive, steering_vector=v, strength=strength))
    with model.hooks(fwd_hooks=[fwd_hook]):
        out = model.generate(
            tokens,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
        )
    return model.tokenizer.decode(out[0])

prompt = "I am feeling good"
print("Baseline:  ", model.generate(prompt, max_new_tokens=64, temperature=0.5))
print("Steered +1:", steer_simple_add(prompt, strength=5.0))
print("Steered -1:", steer_simple_add(prompt, strength=-5.0))


Loaded pretrained model gpt2-small into HookedTransformer


  0%|          | 0/64 [00:00<?, ?it/s]

Baseline:   I am feeling good. I am feeling good. I am feeling good. I am feeling good. I am feeling good. I am feeling good. I am feeling good. I am feeling good. I am feeling good. I am feeling good. I am feeling good. I am feeling good. I am feeling good. I am feeling


  0%|          | 0/64 [00:00<?, ?it/s]

Steered +1: <|endoftext|>I am feeling good about my life. I am feeling good about myself. I am feeling good about everyone I know. I am feeling good about myself. I am feeling good about myself. I am feeling good about myself. I am feeling good about myself. I am feeling good about myself. I am feeling good about myself. I am


  0%|          | 0/64 [00:00<?, ?it/s]

Steered -1: <|endoftext|>I am feeling good about my life. I am feeling good about myself. I am feeling good about everyone I know. And I am feeling good about myself. I am feeling good about myself.

I am feeling good about myself. I am feeling good about myself. I am feeling good about myself. I am feeling good about myself


# FROM SAELENS SOURCE CODE

In [2]:
# from transformer_lens import HookedTransformer
from sae_lens import SAE, HookedSAETransformer

# model = HookedSAETransformer.from_pretrained("gpt2-small", device="cuda")
# sae, sae_cfg, _ = SAE.from_pretrained(
#     release="gpt2-small-res-jb",  # <- Release name
#     sae_id="blocks.7.hook_resid_pre",  # <- SAE id (not always a hook point!)
#     device="cuda",
# )

# ==========================================================================

device = "cuda"
MODEL_NAME = "gemma-2-2b"

SAE_TYPE = "res"    # res/ mlp/ att
LAYER = 25
SAE_ID = f"layer_{LAYER}/width_16k/canonical"
RELEASE = f"gemma-scope-2b-pt-{SAE_TYPE}-canonical"


model = HookedSAETransformer.from_pretrained(MODEL_NAME, device="cuda")
sae, cfg, _ = SAE.from_pretrained(
    release=RELEASE,  # <- Release name
    sae_id=SAE_ID,  # <- SAE id (not always a hook point!)
    device="cuda",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


In [4]:
# Generate text without steering for comparison
prompt = "I am feeling good"

def generate_without_steering(model, sae, prompt, max_new_tokens):
    normal_text = model.generate(
        prompt,
        max_new_tokens=max_new_tokens,
        stop_at_eos=False if model.cfg.device == "mps" else True,
        prepend_bos=sae.cfg.prepend_bos,
    )
    return normal_text

  0%|          | 0/65 [00:00<?, ?it/s]

I am feeling good about winning the recent National Capital Comic-Con. The announcement was made on May 6, and I have been feeling a little under the weather since then. All I can say: I was so sick after the National Capital event, I had trouble pronouncing the word CHEESE. I thought I had CHEESA. I’


In [7]:
from functools import partial


def steering(
    activations, hook, steering_strength=1.0, steering_vector=None, max_act=1.0
):
    # Note if the feature fires anyway, we'd be adding to that here.
    return activations + max_act * steering_strength * steering_vector


def generate_with_steering(
    model,
    sae,
    prompt,
    steering_feature,
    max_act,
    steering_strength=1.0,
    max_new_tokens=64,
):
    input_ids = model.to_tokens(prompt, prepend_bos=sae.cfg.prepend_bos)

    steering_vector = sae.W_dec[steering_feature].to(model.cfg.device)

    steering_hook = partial(
        steering,
        steering_vector=steering_vector,
        steering_strength=steering_strength,
        max_act=max_act,
    )

    # standard transformerlens syntax for a hook context for generation
    with model.hooks(fwd_hooks=[(sae.cfg.hook_name, steering_hook)]):
        output = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            temperature=0.5,
            top_p=0.9,
            stop_at_eos=False if device == "mps" else True,
            prepend_bos=sae.cfg.prepend_bos,
        )

    return model.tokenizer.decode(output[0])

# Generate text with steering
steered_text = generate_with_steering(
    model=model, sae=sae, prompt=prompt, steering_feature=1, max_act=1.0, steering_strength=2.0
)
print("Steered text:")
print(steered_text)

  0%|          | 0/64 [00:00<?, ?it/s]

Steered text:
<bos>I am feeling good about the way the last week has gone.  I’ve been working on a lot of things and getting a lot done.  I’ve been doing a lot of reading and research and writing.  I’ve been doing a lot of planning and strategizing.  I’ve been doing a lot of


In [ ]:
print("Without Steering")
print(generate_without_steering(model, sae, prompt, max_new_tokens=64))

In [ ]:
print("Steered text:")
steered_text = generate_with_steering(
    model=model, sae=sae, prompt=prompt, steering_feature=1, max_act=1.0, steering_strength=2.0
)
print(steered_text)

# Test

NameError: name 'model' is not defined

In [1]:
# instantiate an object to hold activations from a dataset
from sae_lens import ActivationsStore

# a convenient way to instantiate an activation store is to use the from_sae method
activation_store = ActivationsStore.from_sae(
    model=model,
    dataset=sae.cfg.metadata.dataset_path,
    sae=sae,
    streaming=True,
    # fairly conservative parameters here so can use same for larger
    # models without running out of memory.
    store_batch_size_prompts=8,
    train_batch_size_tokens=4096,
    n_batches_in_buffer=32,
    device=device,
)

NameError: name 'model' is not defined

In [ ]:
# instantiate an object to hold activations from a dataset
from sae_lens import ActivationsStore

# a convenient way to instantiate an activation store is to use the from_sae method
activation_store = ActivationsStore.from_sae(
    model=model,
    dataset=sae.cfg.metadata.dataset_path,
    sae=sae,
    streaming=True,
    # fairly conservative parameters here so can use same for larger
    # models without running out of memory.
    store_batch_size_prompts=8,
    train_batch_size_tokens=4096,
    n_batches_in_buffer=32,
    device=device,
)

from tqdm import tqdm
from functools import partial
import re


def find_max_activation(model, sae, activation_store, feature_idx, num_batches=100):
    """
    Find the maximum activation for a given feature index. This is useful for
    calibrating the right amount of the feature to add.
    """
    max_activation = 0.0

    pbar = tqdm(range(num_batches))
    for _ in pbar:
        tokens = activation_store.get_batch_tokens()

        layer = int(re.search(r"\.(\d+)\.", sae.cfg.metadata.hook_name).group(1))  # type: ignore
        _, cache = model.run_with_cache(
            tokens,
            stop_at_layer=layer + 1,
            names_filter=[sae.cfg.metadata.hook_name],
        )
        sae_in = cache[sae.cfg.metadata.hook_name]
        feature_acts = sae.encode(sae_in).squeeze()

        feature_acts = feature_acts.flatten(0, 1)
        batch_max_activation = feature_acts[:, feature_idx].max().item()
        max_activation = max(max_activation, batch_max_activation)

        pbar.set_description(f"Max activation: {max_activation:.4f}")

    return max_activation

max_act = find_max_activation(model, sae, activation_store, steering_feature)
